In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [20]:
def scale(x_train, x_test):
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = pd.DataFrame(scaler.transform(x_train), 
                           index=x_train.index, 
                           columns=x_train.columns)

    x_test = pd.DataFrame(scaler.transform(x_test),
                          index=x_test.index, 
                          columns=x_test.columns)
    return x_train, x_test

def transform_label(y):
    labeled_y = y.apply(lambda row: int(row.damage_grade.split('_')[1])-1, 1).values
    return labeled_y

def transform_3label(y):
    labeled_y = y.apply(lambda row: int(row.damage_grade[1])-1, 1).values
    return labeled_y

def create_nn(lr, end_neurons=5):
    model = keras.Sequential()
    model.add(keras.layers.Dense(500, input_shape=(80,), activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(end_neurons, activation='softmax'))

    #stochastic gradient descent
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [4]:
lr=0.05
epochs=15
batch_size=60

nn_model = create_nn(lr)

## 1. 5-classes classification:

### 1.1. Original set

In [4]:
x = pd.read_csv('../Project_data/processed_data/x_post_preproc.csv', index_col='building_id')
y = pd.read_csv('../Project_data/processed_data/y_post_preproc.csv', index_col='building_id')

In [8]:
x_train, x_test = train_test_split(x, test_size=0.2, random_state=42)
y_train, y_test = transform_label(y.loc[x_train.index]), transform_label(y.loc[x_test.index])

In [9]:
x_train_norm, x_test_norm = scale(x_train, x_test)

In [10]:
nn_model.fit(x_train_norm, y_train, 
             epochs=epochs, batch_size=batch_size,
             validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/15
609675/609675 [==============================] - 55s 90us/sample - loss: 1.0615 - accuracy: 0.5492 - val_loss: 1.0482 - val_accuracy: 0.5517
Epoch 2/15
609675/609675 [==============================] - 51s 84us/sample - loss: 1.0310 - accuracy: 0.5622 - val_loss: 1.0317 - val_accuracy: 0.5597
Epoch 3/15
609675/609675 [==============================] - 53s 87us/sample - loss: 1.0197 - accuracy: 0.5670 - val_loss: 1.0317 - val_accuracy: 0.5567
Epoch 4/15
609675/609675 [==============================] - 48s 79us/sample - loss: 1.0116 - accuracy: 0.5699 - val_loss: 1.0246 - val_accuracy: 0.5615
Epoch 5/15
609675/609675 [==============================] - 48s 78us/sample - loss: 1.0051 - accuracy: 0.5727 - val_loss: 1.0164 - val_accuracy: 0.5666
Epoch 6/15
609675/609675 [==============================] - 48s 79us/sample - loss: 0.9993 - accuracy: 0.5756 - val_loss: 1.0247 - val_accuracy: 0.5624
Epoch 7/15
609675/609675 [==========

In [20]:
preds = nn_model.predict_proba(x_test_norm)

In [22]:
pd.DataFrame(preds).to_csv('../Project_data/results/nn_preds_orig.csv')

### 1.2 Resampled set:

#### 1.2.1 Over-sampling with SMOTENC

In [23]:
x_train = pd.read_csv('../Project_data/processed_data/resampling/x_train_smotenc.csv', index_col=0).reset_index(drop=True)
y_train = pd.read_csv('../Project_data/processed_data/resampling/y_train_smotenc.csv', index_col=0).reset_index(drop=True)

C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [24]:
y_train = transform_label(y_train)

In [25]:
x_train_norm, x_test_norm = scale(x_train, x_test)

In [26]:
nn_model.fit(x_train_norm, y_train, 
             epochs=epochs, batch_size=batch_size,
             validation_data=(x_test_norm, y_test))

Train on 1104100 samples, validate on 152419 samples
Epoch 1/15
1104100/1104100 [==============================] - 98s 88us/sample - loss: 0.9287 - accuracy: 0.6050 - val_loss: 1.0216 - val_accuracy: 0.5671
Epoch 2/15
1104100/1104100 [==============================] - 92s 84us/sample - loss: 0.9015 - accuracy: 0.6146 - val_loss: 1.0176 - val_accuracy: 0.5671
Epoch 3/15
1104100/1104100 [==============================] - 88s 80us/sample - loss: 0.8899 - accuracy: 0.6193 - val_loss: 1.0203 - val_accuracy: 0.5672
Epoch 4/15
1104100/1104100 [==============================] - 88s 80us/sample - loss: 0.8825 - accuracy: 0.6221 - val_loss: 1.0174 - val_accuracy: 0.5694
Epoch 5/15
1104100/1104100 [==============================] - 91s 82us/sample - loss: 0.8769 - accuracy: 0.6242 - val_loss: 1.0099 - val_accuracy: 0.5715
Epoch 6/15
1104100/1104100 [==============================] - 103s 93us/sample - loss: 0.8725 - accuracy: 0.6264 - val_loss: 1.0066 - val_accuracy: 0.5728
Epoch 7/15
1104100/110

In [27]:
preds = nn_model.predict(x_test_norm)

In [28]:
pd.DataFrame(preds).to_csv('../Project_data/results/nn_preds_smotenc.csv')

### 1.2.2 Under-sampling: cleaning oversampled dataset

#### 1.2.2.1 Tomek

In [29]:
x_train = pd.read_csv('../Project_data/processed_data/resampling/x_train_smotenc_tmk.csv', index_col=0).reset_index(drop=True)
y_train = pd.read_csv('../Project_data/processed_data/resampling/y_train_smotenc_tmk.csv', index_col=0).reset_index(drop=True)

In [30]:
y_train = transform_label(y_train)

In [31]:
x_train_norm, x_test_norm = scale(x_train, x_test)

In [32]:
nn_model.fit(x_train_norm, y_train,
             epochs=epochs, batch_size=batch_size,
             validation_data=(x_test_norm, y_test))

Train on 897616 samples, validate on 152419 samples
Epoch 1/15
897616/897616 [==============================] - 73s 82us/sample - loss: 0.8221 - accuracy: 0.6497 - val_loss: 1.0210 - val_accuracy: 0.5735
Epoch 2/15
897616/897616 [==============================] - 77s 85us/sample - loss: 0.8188 - accuracy: 0.6513 - val_loss: 1.0308 - val_accuracy: 0.5683
Epoch 3/15
897616/897616 [==============================] - 76s 85us/sample - loss: 0.8165 - accuracy: 0.6522 - val_loss: 1.0292 - val_accuracy: 0.5727
Epoch 4/15
897616/897616 [==============================] - 75s 84us/sample - loss: 0.8143 - accuracy: 0.6535 - val_loss: 1.0304 - val_accuracy: 0.5721
Epoch 5/15
897616/897616 [==============================] - 77s 85us/sample - loss: 0.8124 - accuracy: 0.6543 - val_loss: 1.0274 - val_accuracy: 0.5692
Epoch 6/15
897616/897616 [==============================] - 76s 85us/sample - loss: 0.8105 - accuracy: 0.6550 - val_loss: 1.0335 - val_accuracy: 0.5662
Epoch 7/15
897616/897616 [==========

In [33]:
preds = nn_model.predict(x_test_norm)

In [35]:
pd.DataFrame(preds).to_csv('../Project_data/results/nn_preds_smotenc_tmk.csv')

#### 1.2.2.2 ENN

In [36]:
x_train = pd.read_csv('../Project_data/processed_data/resampling/x_train_smotenc_enn.csv', index_col=0).reset_index(drop=True)
y_train = pd.read_csv('../Project_data/processed_data/resampling/y_train_smotenc_enn.csv', index_col=0).reset_index(drop=True)

In [37]:
y_train = transform_label(y_train)

In [38]:
x_train_norm, x_test_norm = scale(x_train, x_test)

In [39]:
nn_model.fit(x_train_norm, y_train, 
             epochs=epochs, batch_size=batch_size,
             validation_data=(x_test_norm, y_test))

Train on 526650 samples, validate on 152419 samples
Epoch 1/15
526650/526650 [==============================] - 48s 91us/sample - loss: 0.7151 - accuracy: 0.6968 - val_loss: 1.1118 - val_accuracy: 0.5532
Epoch 2/15
526650/526650 [==============================] - 46s 87us/sample - loss: 0.7052 - accuracy: 0.7009 - val_loss: 1.1119 - val_accuracy: 0.5553
Epoch 3/15
526650/526650 [==============================] - 47s 90us/sample - loss: 0.7000 - accuracy: 0.7035 - val_loss: 1.1234 - val_accuracy: 0.5507
Epoch 4/15
526650/526650 [==============================] - 48s 91us/sample - loss: 0.6963 - accuracy: 0.7057 - val_loss: 1.1433 - val_accuracy: 0.5493
Epoch 5/15
526650/526650 [==============================] - 48s 90us/sample - loss: 0.6930 - accuracy: 0.7067 - val_loss: 1.1436 - val_accuracy: 0.5501
Epoch 6/15
526650/526650 [==============================] - 49s 93us/sample - loss: 0.6899 - accuracy: 0.7078 - val_loss: 1.1637 - val_accuracy: 0.5491
Epoch 7/15
526650/526650 [==========

In [40]:
preds = nn_model.predict(x_test_norm)

In [41]:
pd.DataFrame(preds).to_csv('../Project_data/results/nn_preds_smotenc_enn.csv')

## 2. 3-classes classification:

In [21]:
nn_model = create_nn(lr, end_neurons=3)

In [5]:
x = pd.read_csv('../Project_data/processed_data/x_post_preproc.csv', index_col='building_id')
y = pd.read_csv('../Project_data/processed_data/y_post_preproc_3lab.csv', index_col='building_id')

In [6]:
x_train, x_test = train_test_split(x, test_size=0.2, random_state=42)
y_train, y_test = transform_3label(y.loc[x_train.index]), transform_3label(y.loc[x_test.index])

In [16]:
x_train_norm, x_test_norm = scale(x_train, x_test)

In [24]:
nn_model.fit(x_train_norm, y_train, 
             epochs=epochs, batch_size=batch_size,
             validation_data=(x_test_norm, y_test))

Train on 609675 samples, validate on 152419 samples
Epoch 1/15
609675/609675 [==============================] - 41s 68us/sample - loss: 0.6779 - accuracy: 0.6969 - val_loss: 0.6798 - val_accuracy: 0.6960
Epoch 2/15
609675/609675 [==============================] - 41s 67us/sample - loss: 0.6710 - accuracy: 0.7002 - val_loss: 0.6724 - val_accuracy: 0.6995curacy: 0.70
Epoch 3/15
609675/609675 [==============================] - 40s 65us/sample - loss: 0.6661 - accuracy: 0.7028 - val_loss: 0.6718 - val_accuracy: 0.6988
Epoch 4/15
609675/609675 [==============================] - 40s 65us/sample - loss: 0.6619 - accuracy: 0.7042 - val_loss: 0.6722 - val_accuracy: 0.6987
Epoch 5/15
609675/609675 [==============================] - 41s 67us/sample - loss: 0.6580 - accuracy: 0.7056 - val_loss: 0.6890 - val_accuracy: 0.6901
Epoch 6/15
609675/609675 [==============================] - 41s 68us/sample - loss: 0.6549 - accuracy: 0.7072 - val_loss: 0.6678 - val_accuracy: 0.7022
Epoch 7/15
609675/609675

In [25]:
preds = nn_model.predict(x_test_norm)

In [28]:
pd.DataFrame(preds).to_csv('results/nn_preds_3cls.csv')